# HOW TO ORGANIZE PYTORCH INTO LIGHTNING
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/starter/converting.html)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512), 21.7.13)
코드가 Lightning과 작동할 수 있도록 PyTorch를 Lightning으로 구성하는 방법은 다음과 같습니다.

---


## 1. Computational code 옮기기

모델 아키텍처와 forward pass를 Lightning module로 전달합니다.

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import LightningModule

class LitModel(LightningModule):

    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        return x

---

## 2. Optimizer와 Scheduler 옮기기

옵티마이저를 `configure_optimizers()` hook으로 옮깁니다.


In [ ]:
class LitModel(LightningModule):

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

---

## 3. Train loop 알맹이 찾기

Lightning은 대부분의 Train, Epoch 및 Batch iteration을 자동화합니다. 유지해야 하는 것은 training step 논리 뿐입니다. 이것을 `training_step()` hook으로 이동해야 합니다(이 경우 hook parameters인, batch와 batch_idx를 사용해야 함).

In [ ]:
class LitModel(LightningModule):

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

---

## 4. Val loop 알맹이 찾기

(선택 사항) 검증 루프를 추가하려면 `validation_step()` hook에 논리를 추가하세요(이 경우 hook parameters인, batch와 batch_idx를 사용해야 함).

In [ ]:
class LitModel(LightningModule):

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y)
        return val_loss

> NOTE: `model.eval()`과 `torch.no_grad()`은 Validation 시 자동으로 호출됩니다.

---

## 5. Test loop 알맹이 찾기

(선택 사항) 검증 루프를 추가하려면 `test_step()` hook에 논리를 추가하세요(이 경우 hook parameters인, batch와 batch_idx를 사용해야 함).

In [ ]:
class LitModel(LightningModule):

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

> NOTE: `model.eval()`과 `torch.no_grad()`은 test 시 자동으로 호출됩니다.

테스트는 자동으로 이뤄지지 않습니다.


In [ ]:
trainer.test()

> TIP: `.test()`은 모델의 Best checkpoint를 자동으로 로드합니다.

## 6. `.cuda()`와 `to.device()` 호출 제거하기

[Lightning 모듈](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html)은 어떠한 하드웨어 조건에서도 자동으로 실행될 수 있습니다!